In [4]:
# Using uncalibrated airplane data, generate baseline and antenna calibrations for a given day
#
# Author: Brian Pitzel
# Date Created: 3 March 2025
# Date Modified: 3 March 2025

import numpy as np
import h5py 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import datetime
import csv
import glob
import pickle
import pymap3d as pm
from datetime import timezone
import warnings
import helpers.airplanes as ap
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib widget

vfromtimestamp = np.vectorize(datetime.datetime.fromtimestamp)
vutcfromtimestamp = np.vectorize(datetime.datetime.utcfromtimestamp)

In [5]:
# hard-coded magic numbers needed for the script

# download aircraft data only if it falls within these lat/lon boundarie
# west, south, east, north
SUPERDARN_BOUNDS = (-110.0, 52.1, -101.8, 55.78) # for remington

# convert feet to meters
FT_TO_M = 1 / 3.281

In [6]:
# inputs to the script:
# ymd - date of the day to calibrate on (will use data from all 24 hours unless otherwise specified)
# level1_files - UNCALIBRATED level 1 data (range correction applied, NO rx calibration applied)

import traffic
from traffic.data import opensky

year = 2025
month = 3
day = 13

aircrafts_dbs = []


In [39]:
# first, find all of the airplane echoes in the level1_data.
# this allows us to pull airplane data at the proper times, instead of pulling ALL data from the entire day
hour = 18
s_minute = 30
e_minute = 45
t_start = datetime.datetime(year, month, day, hour, s_minute, 0, tzinfo=timezone.utc)
t_end = datetime.datetime(year, month, day, hour, e_minute, 0, tzinfo=timezone.utc)

# get aircraft data for the airplane timeframe
aircrafts_db = opensky.history(
                    t_start,
                    t_end,
                    bounds=SUPERDARN_BOUNDS)

aircrafts_dbs.append(aircrafts_db)   

FINISHED: : 100% [00:01, 67.9%/s]
DOWNLOAD: 6.29klines [00:00, 226klines/s]


In [40]:
aircrafts_dbs

[   icao24 callsign  count
 0  39c495   FBU80G    301
 1  a4e714   DAL142    301
 2  c01460   LVW357    301
 3  c03001   ACA305    301
 4  c044c8    RS270    301
 5  c07445     SLG3    301
 6  c08620    CNK22    301
 7  4bb142   THY4MS    283
 8  c037ab    CFVCC    229
 9  c02294  NCB1254    148,
    icao24 callsign  count
 0  39c495   FBU80G    360
 1  484411   KLM226    360
 2  c01460   LVW357    360
 3  c03001   ACA305    360
 4  c07e59   WJA434    360
 5  4bb142   THY4MS    359
 6  c07445     SLG3    344
 7  c02294  NCB1254    187
 8  a762cc   UPS109    152
 9  c08620    CNK22    131,
    icao24 callsign  count
 0  484411   KLM226    721
 1  a96b14   CKS868    721
 2  c05056   ACA357    721
 3  c07445     SLG3    721
 4  c074ef    CGSHI    589
 5  c08509   ACA850    566
 6  899107  CAL5233    509
 7  847183   NCA134    412
 8  c047e5    RS290    351
 9  06a128   QTR62P    342,
    icao24 callsign  count
 0  3c4b33   DLH455    301
 1  3c5c48    BTX8H    301
 2  71c075   KAL269    30

In [41]:
ap.save_aircrafts_dbs(aircrafts_dbs, f'superdarn_{year:04d}-{month:02d}-{day:02d}')